# API de Transporte Público Ciudad de Buenos Aires

#### ¿Qué es una API (*application programming interface*)?
Una API es un conjunto de funciones predefinidas que se utilizan para el desarrollo de aplicaciones, ofrecidas por una biblioteca para ser utilizadas por otro software. Uno de sus principales propósitos consiste en proporcionar funciones de uso general de forma que los desarrolladores hagan uso de las mismas para evitar escribir código desde cero.

#### La API de Transporte Público de Buenos Aires
Tiene por objetivo proveer los datos abiertos del transporte público de la Ciudad en formato estandarizado y en **tiempo real**.

Algunos de los conjuntos de datos centrales multimodales incluidos y disponibles para los desarrolladores son:

> - Planificación del viaje (actual y futuro)
> - Estado (actual y futuro)
> - Interrupciones (actuales) y trabajos planificados (futuro)
> - Predicciones de llegada / salida (instantánea y websockets)
> - Horarios
> - Puntos de embarque e instalaciones
> - Rutas y líneas (topología y geográfica)



### Paso 1: Importar las Bibliotecas necesarias


In [1]:
PENDIENTE ## Explicar para que sirve cada una.

In [2]:
import pandas as pd
import numpy as np
import re
import requests
from pandas.io.json import json_normalize
import json
import datetime
import urllib.request
from time import sleep
import time
!pip install plotly
import plotly.express as px


### Construcción de la URL a consultar

> **1-** Ingreso a *https://www.buenosaires.gob.ar/desarrollourbano/transporte/apitransporte/api-doc*

> **2-** Selecciono la opción */colectivos/vehiclePositions*: Devuelve la posición de los vehículos monitoreados actualizada cada 30 segundos 

> **3-** Ingreso las credenciales "client_id" y "client_secret" y el parámetro "agency_id" (para seleccionar la línea de colectivos que me interesa)

> **4-** Selecciono "GET" y copio la URL que se construye.

Creo un objeto llamado url de tipo STR con la url que copié en la documentación de la API.

Notar que se construye concatenando la consulta que nos interesa **"vehiclePositionsSimple"** con el parámetro **"agency_id"** y las credenciales **"client_id"** y **"client_secret"**

In [4]:
url='https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple?agency_id=6&client_id=a0119e3e5ebf4fb4a0cbdea04de12037&client_secret=19967Bb82e7e4C7bab2CA49e3d631C3E' 

### Alternativa para construir la url

Creo seteo el parametro que me interesa y las credenciales de la API en objetos del tipo STR para luego completar las URL que se encuentra en otro objeto del tipo STR


In [5]:
# Option 2: Create a function
def url_bus(id_agency, user, psw):
    
    url= 'https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple?agency_id='+str(id_agency)+'&client_id='+str(user)+'&client_secret='+str(psw) 
    
    return url    

In [6]:
# Option 3: Set parameters & credentials in objects

agency_id= '6'

client_id= 'a0119e3e5ebf4fb4a0cbdea04de12037'

client_secret= '19967Bb82e7e4C7bab2CA49e3d631C3E'

url_incompleta= 'https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple?agency_id={}&client_id={}&client_secret={}' 

# Con '.format' agrego otros strings al string principal 'url' 
# agregando como parametros lo que quiero que aparezca entre los corchetes

url_buses= url.format(agency_id,
                      client_id,
                      client_secret)

## Loop para recolectar los datos de la API en un período de tiempo

In [9]:
# Create an empty Data Frame
linea_12=pd.DataFrame()

# this pattern captures all text in {}
pattern='({.*?})'

# 60 sec * 10 min
timeout = time.time() + 60*10
 
while True:
    
    # API requests
    response= requests.get(url)
    
    # Get text from API response
    text_api= response.text
    
    # Capture text in {} from the API response
    data_buses=re.findall(pattern, text_api.replace('\n',''))
   
    # For loop to append data from all bus units to empty Data Frame
    for i in range(len(data_buses)):
        
        # Convert to JSON (or dict) each row of the API response text 
        json_buses=json.loads(data_buses[i])
        
        # Convert to DataFrame and append to linea_12 DF 
        linea_12=linea_12.append(json_normalize(json_buses), ignore_index=True)
       
    print(linea_12)
    
    if time.time() > timeout:
        
        print('Fin del tiempo de ejecución')
        
        break
        
    ## Wait 30 seconds to get new data from the API               
    sleep(30)
    

    agency_id                          agency_name  direction     id  \
0           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10          6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11          6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12          6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   

     agency_id                          agency_name  direction     id  \
0            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12           6  TRANSPORTES AUTOMOTORES CALLAO S.A.

     agency_id                          agency_name  direction     id  \
0            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12           6  TRANSPORTES AUTOMOTORES CALLAO S.A.

     agency_id                          agency_name  direction     id  \
0            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12           6  TRANSPORTES AUTOMOTORES CALLAO S.A.

     agency_id                          agency_name  direction     id  \
0            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11           6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12           6  TRANSPORTES AUTOMOTORES CALLAO S.A.

...

      agency_id                          agency_name  direction     id  \
0             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12            6  TRANSPORTES AUTOMOTOR

      agency_id                          agency_name  direction     id  \
0             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12            6  TRANSPORTES AUTOMOTOR

      agency_id                          agency_name  direction     id  \
0             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12            6  TRANSPORTES AUTOMOTOR

      agency_id                          agency_name  direction     id  \
0             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12            6  TRANSPORTES AUTOMOTOR

      agency_id                          agency_name  direction     id  \
0             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3766   
1             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3789   
2             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3803   
3             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3810   
4             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3811   
5             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3815   
6             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3821   
7             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3839   
8             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3851   
9             6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3857   
10            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          0   3860   
11            6  TRANSPORTES AUTOMOTORES CALLAO S.A.          1   3872   
12            6  TRANSPORTES AUTOMOTOR

## Los datos recolectados de la API estan en linea_12

In [19]:
linea_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 11 columns):
agency_id           1999 non-null int64
agency_name         1999 non-null object
direction           1999 non-null int64
id                  1999 non-null object
latitude            1999 non-null float64
longitude           1999 non-null float64
route_id            1999 non-null object
route_short_name    1999 non-null object
speed               1999 non-null float64
timestamp           1999 non-null int64
trip_headsign       1999 non-null object
dtypes: float64(3), int64(3), object(5)
memory usage: 171.9+ KB


### Análisis del Dataframe construido obteniendo datos de la API

#### Metadatos

>- **'agency_id'**: Número de referencia de la línea

>- **'agency_name'**: Nombre de la empresa que gestiona la línea

>- **'direction'** : Dirección a la que se dirige (Dummy)

>- **'id'**: Id del interno de la línea

>- **'latitude'**: Latitud de la ubicación del colectivo

>- **'longitude'**: Longitud de la ubicación del colectivo

>- **'route_id'**: Identificación de la línea perteneciente a la empresa

>- **'route_short_name'**: Nombre de la línea perteneciente a la empresa

>- **'speed'**: Velocidad

>- **'timestamp'**: Hora de la consulta (en S)

>- **'trip_headsign'**: Destino anunciado del viaje

In [39]:
# Parseo la columna timestamp
linea_12['timestamp']=pd.Series([pd.Timestamp(t, unit='s') for t in linea_12['timestamp']])

In [44]:
# Filtro los datos de la linea 12
linea_12A=linea_12[linea_12['route_id']=='63']

In [47]:
linea_12A=linea_12A.reset_index(drop=True)
# cuento la cantidad de registros de cada uno de los internos
linea_12A['id'].value_counts()

3821     21
3894     21
3958     21
4034     21
19988    21
3860     21
3946     21
3990     21
3918     21
4019     21
25686    21
25503    21
3949     21
3947     21
4037     21
3984     21
3811     21
4033     21
36876    21
3879     21
3929     21
4822     21
3923     21
3928     21
3983     21
4032     21
3878     21
3972     21
3988     21
4076     21
3974     21
3911     21
3910     21
3857     21
16404    21
3970     21
3803     21
4038     21
3900     21
3965     21
3851     21
19771    21
3766     20
3997     20
4021     20
3882     19
4214     19
4111     18
19760    17
3863     17
3810     16
3956     15
3815     15
3839     15
3872     14
3789     13
3767     12
3846     11
3942      6
4009      2
Name: id, dtype: int64

In [50]:
#filtro el interno 3821
interno_3821=linea_12A[linea_12A['id']=='3821']     

## graficar el recorrido de los colectivos

In [1]:
## Grafico todos los internos de la L 12 conectando los puntos lat long con lineas para simular recorridos

In [77]:

fig = px.line_mapbox(linea_12A, lat="latitude", lon="longitude", color="id", zoom=3, height=300)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=11, mapbox_center_lat = -34.6,
    margin={"r":1,"t":0,"l":0,"b":0})

fig.show()

In [2]:
## Grafico todos los internos de la L 12 con puntos lat long

In [78]:
fig = px.scatter_mapbox(linea_12A, lat="latitude", lon="longitude", color="id", zoom=3, height=300)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=11, mapbox_center_lat = -34.6,
    margin={"r":1,"t":0,"l":0,"b":0})

fig.show()
